<a href="https://colab.research.google.com/github/GomideB/ExemploDoc/blob/main/OFICIAL_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- CÉLULA 1: INSTALAÇÃO E REINÍCIO DO AMBIENTE ---
# Esta célula deve ser executada sozinha. Após a execução, o Colab irá reiniciar o kernel.
# ESPERE o Colab reiniciar ANTES de executar a próxima célula.

# Instalação de bibliotecas essenciais para o projeto.
# - 'google-genai': Para interagir com os modelos Gemini (Google AI Studio).
# - 'google-adk': Google Agent Development Kit, que facilita a construção de agentes de IA.
# - 'aiohttp': Para fazer requisições HTTP assíncronas, crucial para as chamadas à API do Google Maps.
%pip -q install google-genai google-adk aiohttp

In [3]:
# --- Configurações Iniciais e Importações ---
import os
import asyncio
import aiohttp
import google.genai
from google.colab import userdata
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from typing import List, Dict, Optional
import nest_asyncio
import textwrap
from IPython.display import display, Markdown, HTML

# --- INÍCIO DA SUPRESSÃO DE LOGS INDESEJADOS ---
import logging
import warnings

# Suprime avisos e erros do logger 'asyncio' (e.g., 'Task was destroyed but it is pending!')
logging.getLogger('asyncio').setLevel(logging.CRITICAL)

# Suprime avisos e erros do logger 'httpx' (usado internamente por google-genai/adk para requisições HTTP)
logging.getLogger('httpx').setLevel(logging.CRITICAL)

# Suprime outros avisos gerais do Python (e.g., de bibliotecas)
warnings.filterwarnings("ignore")
# --- FIM DA SUPRESSÃO DE LOGS INDESEJADOS ---

# 'nest_asyncio.apply()' permite aninhar loops de evento asyncio,
# o que é necessário para rodar código assínrono em ambientes como o Google Colab,
# onde um loop de evento já pode estar ativo.
nest_asyncio.apply()

# Restante do seu código continua...

In [4]:
# Configuração das chaves de API.
# 'userdata.get()' é uma função do Colab para acessar segredos de forma segura.
# Certifique-se de que 'GOOGLE_API_KEY' e 'Maps_API' estão configuradas nas "Secrets" do Colab.
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_MAPS_API"] = userdata.get('GOOGLE_MAPS_API')
Maps_API_KEY = os.environ["GOOGLE_MAPS_API"] # Chave da API do Google Maps para uso nas funções.

# Inicialização do cliente Gemini.
# Isso deve ser feito uma única vez no início do script.
client = google.genai.Client()

In [5]:
# --- Funções Auxiliares Comuns ---
def executar_agente(agent: Agent, mensagem: str) -> str:
    """
    Função auxiliar genérica para interagir com qualquer agente de IA.
    Ela encapsula a lógica de criação de sessão, execução do agente e coleta da resposta final,
    reduzindo repetição de código.

    Args:
        agent (Agent): A instância do agente ADK a ser executada.
        mensagem (str): A mensagem de entrada para o agente.

    Returns:
        str: A resposta final gerada pelo agente.
    """
    # Cria um serviço de sessão em memória para manter o contexto da conversa com o agente.
    session_service = InMemorySessionService()
    # Cria uma sessão específica para o usuário e o aplicativo (agente).
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um 'Runner' para executar o agente com a sessão configurada.
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Prepara o conteúdo da mensagem para o agente.
    content = types.Content(role="user", parts=[types.Part(text=mensagem)])
    resposta = ""
    # Itera sobre os eventos retornados pelo runner para coletar a resposta final do agente.
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response(): # Verifica se o evento contém a resposta final.
            for part in event.content.parts:
                if part.text: # Concatena todas as partes de texto da resposta.
                    resposta += part.text + "\n"
    return resposta.strip() # Retorna a resposta sem espaços em branco extras no final.

def to_markdown(text: str) -> Markdown:
    """
    Função auxiliar para formatar e exibir texto em Markdown no ambiente do Google Colab.
    Substitui bullet points comuns por asteriscos para renderização correta em Markdown.

    Args:
        text (str): O texto a ser formatado.

    Returns:
        Markdown: Um objeto Markdown para exibição no Colab.
    """
    text = text.replace('•', ' *') # Padroniza bullet points.
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # Adiciona indentação visual de citação.

In [6]:
# --- Definição e Instanciação Única dos Agentes ---
# Cada agente é definido com um nome, modelo de linguagem, instruções específicas
# e uma descrição de sua função.

# Agente 1: Triagem Inicial
agente_triagem = Agent(
    name="agente_triagem",
    model="gemini-2.0-flash",
    instruction="""Você é um assistente de primeiros socorros.
    Pergunte ao usuário sobre o que ele está sentindo para entender a situação de emergência.
    Mantenha a conversa breve e foque em obter uma descrição clara dos sintomas.
    Após a descrição inicial, **pergunte:** "Você sente mais alguma dor ou algo diferente que eu deva saber?"
    """,
    description="Agente para coletar os sintomas iniciais do usuário."
)

In [7]:
# Agente 2: Coleta e Consolidação de Sintomas Adicionais
agente_coleta_sintomas_adicionais = Agent(
    name="agente_coleta_sintomas_adicionais",
    model="gemini-2.0-flash",
    instruction="""Você é um assistente de saúde.
    Sua tarefa é consolidar informações de sintomas. Receberá um resumo dos sintomas iniciais
    e uma descrição de sintomas adicionais fornecidos pelo usuário.
    Crie uma descrição única, clara e completa de TODOS os sintomas do paciente,
    sem incluir nenhuma pergunta ou instrução adicional. Se o usuário indicou que não há
    sintomas adicionais, apenas forneça o resumo dos sintomas iniciais.
    """,
    description="Agente para coletar e consolidar sintomas adicionais do usuário."
)

In [8]:
# Agente 3: Busca de Primeiros Socorros
agente_busca_primeiros_socorros = Agent(
    name="agente_busca_primeiros_socorros",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Utilize a ferramenta de busca do Google para encontrar informações confiáveis e atualizadas
    sobre como tratar a condição médica descrita. Foque em fontes oficiais de saúde (ministérios,
    hospitais, organizações de primeiros socorros). Limite a busca a 5 resultados mais relevantes.
    """,
    description="Agente para buscar informações de primeiros socorros no Google.",
    tools=[google_search] # Este agente utiliza a ferramenta de busca do Google.
)

In [9]:
# Agente 4: Planejamento de Ações
agente_planejamento_acoes = Agent(
    name="agente_planejamento_acoes",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise os resultados da busca do Google fornecidos e crie um plano conciso e lógico de
    ações de primeiros socorros a serem recomendadas ao usuário. O plano deve ser claro,
    objetivo e priorizar as ações mais urgentes. Inclua, se necessário, orientações sobre
    quando procurar ajuda médica profissional e como proceder até lá. Escreva da forma mais direta e enxuta possível.
    """,
    description="Agente para planejar as ações de primeiros socorros com base em resultados de busca."
    # Não possui ferramentas, pois analisa dados já fornecidos.
)

In [10]:
# Agente 5: Avaliação de Emergência
agente_avaliacao_emergencia = Agent(
    name="agente_avaliacao_emergencia",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise a descrição dos sintomas do usuário e o plano de ações de primeiros socorros
    para determinar a gravidade da situação.
    Se a situação for de emergência (e.g., suspeita de infarto, dificuldade respiratória grave,
    perda de consciência), **INSTRUIA O USUÁRIO A LIGAR IMEDIATAMENTE PARA O SERVIÇO DE EMERGÊNCIA (192 - SAMU no Brasil).**
    Caso contrário, oriente o usuário a procurar atendimento médico em um hospital ou clínica.
    Seja extremamente claro e direto na recomendação de emergência.
    """,
    description="Agente para avaliar a gravidade da situação e recomendar a ação apropriada."
)

In [11]:
# Agente 6: Raciocínio de Localização de Ajuda Médica
agente_localizacao_ajuda_medica = Agent(
    name="agente_localizacao_ajuda_medica",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base na localização fornecida e no planejamento criado, você pode raciocinar sobre a necessidade de hospitais e médicos.
    No entanto, a busca real e a geração de rotas serão feitas por funções auxiliares, que te entregarão os dados.
    Apenas indique que a busca por locais próximos está sendo realizada.
    """,
    description="Agente para raciocinar sobre a necessidade de localizar ajuda médica."
)

In [12]:
# Agente 7: Raciocínio de Rota e Navegação
agente_rota_navegacao = Agent(
    name="agente_rota_navegacao",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base nos hospitais encontrados e na sua localização, você pode raciocinar sobre a necessidade de rotas.
    No entanto, a geração de links de rotas será feita por funções auxiliares, que te entregarão os dados.
    Apenas indique que as rotas estão sendo geradas.
    """,
    description="Agente para raciocinar sobre rotas para hospitais."
)

In [13]:
# Agente 8: Orientação Detalhada Final
agente_orientacao_detalhada = Agent(
    name="agente_orientacao_detalhada",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base na **descrição completa dos sintomas, resultados de busca, plano de ações e avaliação de emergência** fornecidos,
    gere instruções gerais e fáceis de seguir para o usuário. Compile todas as informações em um guia claro e acionável.
    Inclua, se possível, links para vídeos do YouTube que demonstrem as técnicas de primeiros socorros recomendadas (pesquise se necessário, mas priorize links relevantes).
    **É fundamental mencionar a importância de procurar ajuda médica profissional imediatamente se a situação for grave,
    e orientar sobre como proceder até a chegada do socorro ou deslocamento para um local de atendimento.**
    Seja conciso, mas completo, fornecendo os próximos passos mais importantes.
    """,
    description="Agente para gerar orientações detalhadas finais de primeiros socorros, compilando todas as informações.",
    tools=[google_search] # Este agente utiliza a ferramenta de busca do Google.
)

In [14]:
# --- Funções Auxiliares para a API do Google Maps (Assíncronas) ---
# Estas funções interagem com a API do Google Places para buscar locais próximos.
# São assíncronas para melhor performance em operações de rede.

async def buscar_hospitais_proximos(latitude: float, longitude: float, raio: int = 50) -> List[Dict]:
    """
    Busca hospitais próximos a uma dada localização usando a API Google Places Nearby Search.
    Limita os resultados a no máximo 4 hospitais para concisão.

    Args:
        latitude (float): Latitude da localização do usuário.
        longitude (float): Longitude da localização do usuário.
        raio (int): Raio de busca em quilômetros. Convertido para metros na URL.

    Returns:
        List[Dict]: Uma lista de dicionários, cada um representando um hospital com nome, endereço, etc.
    """
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=hospital&key={Maps_API_KEY}&language=pt"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url) as response:
                response.raise_for_status() # Lança um erro para respostas HTTP de status 4xx/5xx.
                data = await response.json()
                if data["status"] == "OK":
                    hospitais = []
                    for result in data["results"]:
                        hospitais.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return hospitais[:4] # Limita a 4 hospitais, conforme solicitado.
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados hospitais na área especificada.")
                    return []
                else:
                    print(f"Erro ao buscar hospitais: {data['status']}")
                    return []
        except aiohttp.ClientError as e:
            print(f"Erro de requisição HTTP ao buscar hospitais: {e}")
            return []
        except Exception as e:
            print(f"Erro inesperado ao buscar hospitais: {e}")
            return []

async def buscar_medicos_proximos(latitude: float, longitude: float, raio: int = 50) -> List[Dict]:
    """
    Busca médicos/clínicas próximos a uma dada localização usando a API Google Places Nearby Search.
    Limita os resultados a no máximo 4 médicos/clínicas para concisão.

    Args:
        latitude (float): Latitude da localização do usuário.
        longitude (float): Longitude da localização do usuário.
        raio (int): Raio de busca em quilômetros.

    Returns:
        List[Dict]: Uma lista de dicionários, cada um representando um médico/clínica.
    """
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=doctor&key={Maps_API_KEY}&language=pt"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url) as response:
                response.raise_for_status()
                data = await response.json()
                if data["status"] == "OK":
                    medicos = []
                    for result in data["results"]:
                        medicos.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return medicos[:4] # Limita a 4 médicos, conforme solicitado.
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados médicos na área especificada.")
                    return []
                else:
                    print(f"Erro ao buscar médicos: {data['status']}")
                    return []
        except aiohttp.ClientError as e:
            print(f"Erro de requisição HTTP ao buscar médicos: {e}")
            return []
        except Exception as e:
            print(f"Erro inesperado ao buscar médicos: {e}")
            return []

async def gerar_rotas_Maps(origem_lat: float, origem_long: float, destinos: List[Dict]) -> List[str]:
    """
    Gera URLs para rotas no Google Maps, para navegação.
    Limita os links de rota a no máximo 4.

    Args:
        origem_lat (float): Latitude do ponto de origem (usuário).
        origem_long (float): Longitude do ponto de origem (usuário).
        destinos (List[Dict]): Lista de dicionários de destinos (hospitais/médicos) com suas localizações.

    Returns:
        List[str]: Uma lista de URLs do Google Maps para as rotas.
    """
    rotas = []
    # Itera sobre os primeiros 4 destinos para gerar as rotas.
    for destino in destinos[:4]:
        if "localizacao" in destino and "lat" in destino["localizacao"] and "lng" in destino["localizacao"]:
            destino_lat = destino["localizacao"]["lat"]
            destino_long = destino["localizacao"]["lng"]
            # Cria a URL do Google Maps com origem, destino e modo de viagem.
            # O 'googleusercontent.com' é um domínio confiável do Google para tais redirecionamentos.
            url = f"https://www.google.com/maps/dir/?api=1&origin=?q={origem_lat},{origem_long}&destination={destino_lat},{destino_long}&travelmode=driving"
            rotas.append(url)
        else:
            print(f"Localização inválida para destino: {destino.get('nome', 'N/A')}. Rota não gerada.")
    return rotas[:4] # Garante que apenas 4 rotas sejam retornadas, mesmo se a lista de destinos fosse maior.

In [16]:
# --- Função Orquestrador Principal ---
async def orquestrador_primeiros_socorros(mensagem_usuario: str, latitude_usuario: float, longitude_usuario: float):
    """
    Orquestra a interação entre os diversos agentes de IA e funções auxiliares
    para fornecer assistência de primeiros socorros de forma estruturada.
    A jornada do usuário é guiada através de passos claros e diretos.

    Args:
        mensagem_usuario (str): A descrição inicial dos sintomas fornecida pelo usuário.
        latitude_usuario (float): Latitude da localização atual do usuário.
        longitude_usuario (float): Longitude da localização atual do usuário.
    """
    print("\n" + "="*70)
    print("⚕️ Sistema de Assistência de Primeiros Socorros Ativado ⚕️".center(70))
    print("Siga as orientações para obter a melhor assistência.".center(70))
    print("="*70 + "\n")

    # Passo 1: Triagem Inicial dos Sintomas
    print("\n--- 🗣️ AGENTE DE TRIAGEM: Analisando seus sintomas iniciais... ---\n")
    resposta_triagem = executar_agente(agente_triagem, mensagem_usuario)
    display(to_markdown(f"**Agente de Triagem:** {resposta_triagem}"))
    print("--------------------------------------------------------------")

    # Passo 2: Coleta de Sintomas Adicionais
    # Solicita mais informações para ter um quadro completo, dividindo a pergunta para melhor visualização.
    print("❓ Você sente mais alguma dor ou algo diferente?")
    mensagem_adicional_usuario = input("   (Digite 'não' se não houver): ") # Prompt mais curto para a entrada do usuário

    print("\n--- 🗣️ AGENTE DE SINTOMAS ADICIONAIS: Consolidando as informações... ---\n")
    sintomas_consolidados = executar_agente(agente_coleta_sintomas_adicionais,
                                            f"Resumo dos sintomas iniciais:\n{resposta_triagem}\n\nSintomas adicionais informados pelo usuário:\n{mensagem_adicional_usuario}")
    display(to_markdown(f"**Sintomas Consolidados:**\n{sintomas_consolidados}"))
    print("--------------------------------------------------------------")

    # Passo 3: Busca por Informações de Primeiros Socorros
    # Utiliza a ferramenta de busca para encontrar orientações confiáveis.
    print("\n--- 🔍 AGENTE DE BUSCA: Pesquisando as melhores práticas de primeiros socorros... ---\n")
    resultados_busca = executar_agente(agente_busca_primeiros_socorros, sintomas_consolidados)
    display(to_markdown(f"**Resultados Relevantes da Busca:**\n{resultados_busca}"))
    print("--------------------------------------------------------------")

    # Passo 4: Planejamento das Ações de Primeiros Socorros
    # Com base na busca, formula um plano de ação inicial.
    print("\n--- 📝 AGENTE DE PLANEJAMENTO: Elaborando um plano de ação... ---\n")
    plano_acoes = executar_agente(agente_planejamento_acoes, resultados_busca)
    display(to_markdown(f"**Plano de Ações Sugerido:**\n{plano_acoes}"))
    print("--------------------------------------------------------------")

    # Passo 5: Avaliação da Gravidade da Emergência
    # Determina a urgência da situação e a necessidade de acionar serviços de emergência.
    print("\n--- 🚨 AGENTE DE AVALIAÇÃO DE EMERGÊNCIA: Analisando a gravidade... ---\n")
    recomendacao_emergencia = executar_agente(agente_avaliacao_emergencia, f"Descrição dos sintomas: {sintomas_consolidados}\nPlano de ações: {plano_acoes}")
    display(to_markdown(f"**Avaliação e Recomendação de Emergência:**\n{recomendacao_emergencia}"))
    print("--------------------------------------------------------------")

    # Passo 6: Busca de Locais de Ajuda Médica Próximos
    # Aciona as funções de Maps para encontrar hospitais e médicos.
    print("\n--- 🏥 AGENTE DE LOCALIZAÇÃO: Buscando hospitais e médicos próximos a você... ---\n")
    # O agente apenas sinaliza o que está acontecendo, a busca é feita pelas funções auxiliares.
    executar_agente(agente_localizacao_ajuda_medica, "Coletando localização do usuário e buscando locais de ajuda.")

    hospitais = await buscar_hospitais_proximos(latitude_usuario, longitude_usuario)
    medicos = await buscar_medicos_proximos(latitude_usuario, longitude_usuario)

    print("\n**Hospitais Próximos (Top 4):**")
    if hospitais:
        for hospital in hospitais:
            display(to_markdown(f"- **{hospital.get('nome')}** ({hospital.get('endereco')}, Avaliação: {hospital.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados hospitais próximos."))

    print("\n**Médicos/Clínicas Próximos (Top 4):**")
    if medicos:
        for medico in medicos:
            display(to_markdown(f"- **{medico.get('nome')}** ({medico.get('endereco')}, Avaliação: {medico.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados médicos/clínicas próximos."))
    print("--------------------------------------------------------------")

    # Passo 7: Geração de Rotas para Hospitais
    # Gera links diretos para navegação no Google Maps.
    print("\n--- 🗺️ AGENTE DE NAVEGAÇÃO: Gerando rotas para os locais de ajuda... ---\n")
    # O agente apenas sinaliza o que está acontecendo, a geração é feita pela função auxiliar.
    executar_agente(agente_rota_navegacao, "Gerando links de rotas para os hospitais encontrados.")

    rotas_hospitais = await gerar_rotas_Maps(latitude_usuario, longitude_usuario, hospitais)
    if rotas_hospitais:
        for i, rota in enumerate(rotas_hospitais):
            hospital_nome = hospitais[i].get('nome', 'Destino') if i < len(hospitais) else 'Destino'
            display(to_markdown(f"Rota para **{hospital_nome}**:"))
            display(HTML(f'<a href="{rota}" target="_blank">Clique aqui para a rota</a>'))
    else:
        display(to_markdown("Não foi possível gerar rotas para os hospitais encontrados."))
    print("--------------------------------------------------------------")

    # Passo 8: Orientação Detalhada Final (Síntese Completa)
    # Este agente agora compila todas as informações para a orientação final do usuário.
    print("\n--- ✨ AGENTE DE ORIENTAÇÃO DETALHADA: Compilando seu guia de ação final... ---\n")
    # Prepara um contexto completo para o agente gerar a orientação final.
    contexto_completo_para_orientacao = (
        f"Sintomas consolidados: {sintomas_consolidados}\n\n"
        f"Resultados da busca por primeiros socorros: {resultados_busca}\n\n"
        f"Plano de ações recomendado: {plano_acoes}\n\n"
        f"Avaliação de emergência: {recomendacao_emergencia}\n\n"
        "Locais de ajuda médica encontrados (hospitais e médicos foram listados com endereços e avaliações): "
        f"{'Hospitais: ' + ', '.join([h['nome'] for h in hospitais]) if hospitais else 'Nenhum hospital encontrado'}. "
        f"{'Médicos/Clínicas: ' + ', '.join([m['nome'] for m in medicos]) if medicos else 'Nenhum médico/clínica encontrado'}."
    )
    orientacao_detalhada_final = executar_agente(agente_orientacao_detalhada, contexto_completo_para_orientacao)

    # --- RESUMO FINAL DA ASSISTÊNCIA ---
    # Este é o output consolidado para o usuário, focado na ação imediata.
    print("\n" + "*"*70)
    print("🤖 RESUMO FINAL DA ASSISTÊNCIA DE PRIMEIROS SOCORROS 🤖".center(70))
    print("*"*70 + "\n")

    resposta_final = f"""
**1. Orientações de Primeiros Socorros:**
{textwrap.indent(orientacao_detalhada_final.strip(), '  ')}

**2. Recomendação Urgente:**
{textwrap.indent(recomendacao_emergencia.strip(), '  ')}

---
**IMPORTANTE:** Para detalhes sobre **Hospitais, Médicos/Clínicas e Rotas**, por favor,
**role para cima** e verifique as mensagens anteriores. Os links para o Google Maps estão lá!
"""
    display(Markdown(resposta_final))
    print("\n" + "="*70)
    print("Processo de Assistência Concluído. Fique bem!".center(70))
    print("="*70 + "\n")


# --- Execução Principal do Programa ---
async def main():
    """
    Função principal que inicia o fluxo do sistema de primeiros socorros.
    Coleta a descrição inicial do usuário e uma localização de teste.
    """
    mensagem_usuario = input("❓ Olá! Para iniciar, por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): ")

    # Localização fixa para testes. Em uma aplicação real, você obteria a localização dinâmica do usuário.
    # Exemplo atual: Campinas, SP, Brasil.
    latitude_usuario = -22.9056
    longitude_usuario = -47.0608

    # Chama o orquestrador principal para iniciar a jornada de assistência.
    await orquestrador_primeiros_socorros(mensagem_usuario, latitude_usuario, longitude_usuario)

# Garante que a função 'main' seja executada quando o script é iniciado.
if __name__ == "__main__":
    await main()

❓ Olá! Para iniciar, por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): Dor no ombro

      ⚕️ Sistema de Assistência de Primeiros Socorros Ativado ⚕️      
         Siga as orientações para obter a melhor assistência.         


--- 🗣️ AGENTE DE TRIAGEM: Analisando seus sintomas iniciais... ---



> **Agente de Triagem:** Sinto muito que você esteja sentindo dores no ombro. Você pode descrever a dor com mais detalhes? É uma dor aguda, surda, latejante ou queima? A dor está localizada em um ponto específico ou espalhada por todo o ombro?

--------------------------------------------------------------
❓ Você sente mais alguma dor ou algo diferente?
   (Digite 'não' se não houver): não

--- 🗣️ AGENTE DE SINTOMAS ADICIONAIS: Consolidando as informações... ---



> **Sintomas Consolidados:**
> Dor no ombro.

--------------------------------------------------------------

--- 🔍 AGENTE DE BUSCA: Pesquisando as melhores práticas de primeiros socorros... ---



...

> Rota para **Hospital Beneficência Portuguesa**:

> Rota para **Hospital Santa Elisa**:

--------------------------------------------------------------

--- ✨ AGENTE DE ORIENTAÇÃO DETALHADA: Compilando seu guia de ação final... ---


**********************************************************************
        🤖 RESUMO FINAL DA ASSISTÊNCIA DE PRIMEIROS SOCORROS 🤖         
**********************************************************************




**1. Orientações de Primeiros Socorros:**
  ## Orientações Detalhadas para Dor no Ombro

  Com base nas informações fornecidas, aqui está um guia conciso para lidar com a dor no ombro:

  **1. Avaliação Inicial:**

  *   **Identifique a Causa:** Tente lembrar se houve alguma lesão, movimento repetitivo ou outra causa aparente para a dor.
  *   **Intensidade da Dor:** Avalie a intensidade da dor (leve, moderada, forte).
  *   **Sintomas Associados:** Verifique se há inchaço, vermelhidão, dormência, formigamento ou dificuldade de movimento.

  **2. Primeiros Socorros Imediatos:**

  *   **Repouso:** Evite atividades que agravem a dor.
  *   **Gelo:** Aplique gelo na área afetada por 15-20 minutos, várias vezes ao dia, nas primeiras 48 horas.
  *   **Analgésicos/Anti-inflamatórios:** Considere o uso de medicamentos de venda livre, como paracetamol ou ibuprofeno, para alívio da dor, seguindo as instruções da bula.

  **3. Acompanhamento e Próximos Passos:**

  *   **Monitore a Dor:** Observe se a dor melhora ou piora com as medidas iniciais.
  *   **Compressa Morna:** Após 48 horas, se a inflamação inicial diminuir, pode usar compressa morna para relaxar a musculatura.
  *   **Alongamentos:** Se a dor for leve, tente alongamentos suaves para manter a mobilidade.
  *   **Postura:** Atente-se a manter uma boa postura.

  **4. Quando Procurar Ajuda Médica:**

  *   **Dor Intensa:** Se a dor for muito forte e não melhorar com as medidas iniciais.
  *   **Incapacidade de Movimento:** Se não conseguir mover o braço ou o ombro.
  *   **Inchaço/Deformidade:** Se houver inchaço, vermelhidão ou deformidade visível no ombro.
  *   **Outros Sintomas:** Se a dor for acompanhada de febre, dormência ou formigamento.
  *   **Persistência:** Se a dor persistir por mais de alguns dias, mesmo com tratamento em casa.

  **Vídeos Úteis:**

  Para aprender mais sobre como aplicar gelo e fazer alongamentos suaves, você pode consultar os seguintes vídeos:

  *   **Como aplicar gelo:**
    
  *   **Alongamentos:**
    
  *   **Alongamentos para dor no ombro:** [https://www.youtube.com/watch?v=JoLwi8yJPCE](https://www.youtube.com/watch?v=JoLwi8yJPCE)

  **Importante:**

  *   Esta orientação é para primeiros socorros. Um diagnóstico preciso e um plano de tratamento adequado devem ser fornecidos por um profissional de saúde.
  *   **Não hesite em procurar ajuda médica imediatamente se a dor for intensa, se você não conseguir mover o braço, ou se houver inchaço/deformidade no ombro.**

  **Onde procurar ajuda:**

  *   Você pode procurar ajuda nos hospitais: Hospital Vera Cruz, Real Sociedade Portuguesa de Beneficência, Hospital Beneficência Portuguesa, Hospital Santa Elisa ou nos médicos/clínicas: Clínica Liberman, Clínica de Cardiologia e Reabilitação, Eletroneuromiografia - Dra. Terezinha Prince, S'T Anna Clínica de Cirurgia Plástica S/C Ltda.

  Espero que estas orientações sejam úteis!

**2. Recomendação Urgente:**
  Com base na sua descrição e no plano de ações, a situação **não aparenta ser uma emergência**.

  Recomendo que siga o plano de primeiros socorros que você descreveu e procure atendimento médico em um hospital ou clínica caso a dor não melhore ou piore.

---
**IMPORTANTE:** Para detalhes sobre **Hospitais, Médicos/Clínicas e Rotas**, por favor,
**role para cima** e verifique as mensagens anteriores. Os links para o Google Maps estão lá!



            Processo de Assistência Concluído. Fique bem!             

